# 연습

- 교수님이 올려주신 파일 기반으로 연습

In [1]:
import numpy as np
import scipy.stats as sst

def bsprice(s, k, r, q, t, sigma, flag):
    d1 = (np.log(s/k) + (r - q + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    callOrPut = 1 if flag.lower()=='call' else -1
    nd1 = sst.norm.cdf(callOrPut*d1)
    nd2 = sst.norm.cdf(callOrPut*d2)
    price = callOrPut*(s*np.exp(-q*t)*nd1 - k*np.exp(-r*t)*nd2)
    return price

In [5]:
def mcpvs(s,k,r,q,t,sigma,nsim,flag):
    z = np.random.randn(nsim)
    st = s*np.exp((r-q-0.5*sigma**2)*t + sigma*np.sqrt(t)*z)
    callOrPut = 1 if flag.lower()=='call' else -1    
    payoff = np.maximum(callOrPut*(st-k), 0)    
    pvs = np.exp(-r*t)*payoff    
    return pvs

In [9]:
s = 100
k = 100
r = 0.03
q = 0.01
t = 0.25
sigma = 0.2
flag = 'put'

#Analytic Formula
price = bsprice(s,k,r,q,t,sigma,flag)
print(f"   Price = {price:0.6f}")

#Monte-Carlo Simulation
nsim = 1000000
mc_pvs = mcpvs(s,k,r,q,t,sigma,nsim,flag)
mc_price = mc_pvs.mean()
print(f"MC Price = {mc_price:0.6f}")

   Price = 3.724086
MC Price = 3.726878


In [13]:
mc_pvs.std()

5.322780510872322

In [17]:
conf_level = 0.95

conf_interval = sst.t.interval(
    conf_level, 
    df=nsim-1, # degree of freedom
    loc=mc_price, # location = mean
    scale=mc_pvs.std()/np.sqrt(nsim) # se of the mean
    )
conf_interval

(3.7164457304713023, 3.7373106719233586)

In [18]:
conf_min, conf_max = conf_interval

mc_pvs[(mc_pvs >= conf_min) & (mc_pvs <= conf_max)].size / nsim

0.000793